In [1]:
import torchaudio as ta
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

import tensorflow as tf

from torchsummary import summary
from torchsummaryX import summary as summaryX
from pytorch_model_summary import summary as summarypt
# import pytorch_lightning as pl
import numpy as np
import os

import librosa

import IPython.display as ipd

import numpy as np

import math
import tensorflow as tf

import pandas as pd

import glob

from tqdm.auto import tqdm

from Noise_Reduction_Datagen import Signal_Synthesis_DataGen

import warnings

warnings.filterwarnings("ignore")

import gc

# from numba import jit

In [2]:
noise_dir = "./dataset/UrbanSound8K/all_files/"
signal_dir = "./dataset/cv-corpus-5.1-2020-06-22/en/clips/"
signal_nums_save = "./dataset_loader_files/signal_paths_nums_save.npy"
num_noise_samples=200
num_signal_samples = 100
noise_save_path = "./dataset_loader_files/noise_paths_save.npy"
n_fft=400
win_length=n_fft
hop_len=n_fft//4
create_specgram = False
perform_stft = False
default_sr = 16000
sec = 6
augment=True

signal_synthesis_dataset = Signal_Synthesis_DataGen(noise_dir, signal_dir, \
                signal_nums_save=signal_nums_save, num_noise_samples=num_noise_samples, \
                num_signal_samples=num_signal_samples, noise_path_save=noise_save_path, \
                 n_fft=n_fft, win_length=win_length, hop_len=hop_len, create_specgram=create_specgram, \
                 perform_stft=perform_stft, normalize=True, default_sr=default_sr, sec=sec, augment=False)

Loading noise from saved file
Loading nums from npy file
100


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 16
shuffle = False
num_workers = 14
pin_memory = False
# data_loader = DataLoader(signal_synthesis_dataset, batch_size=BATCH_SIZE, shuffle=shuffle, num_workers=num_workers)
data_loader = DataLoader(signal_synthesis_dataset, batch_size=BATCH_SIZE, shuffle=shuffle, num_workers=num_workers, pin_memory=pin_memory)
# data_loader.to(device)

In [4]:
signal_synthesis_dataset.__len__()

20000

In [5]:
# %%time
# # data_loader_iter = iter(data_loader)
# for index, i in enumerate(data_loader):
# #     i = next(data_loader)
#     if index < 32-1:
#         pass
#     else:
#         break
#     print(i[0].shape, index)

In [6]:
np.floor(((default_sr*sec) - (win_length - 1) - 1)/ hop_len + 3)

959.0

In [7]:
class InstantLayerNormalization(nn.Module):
    def __init__(self, in_shape, out_shape):
        self.in_shape = in_shape
        self.out_shape = out_shape
        
        self.epsilon = 1e-7
        self.gamma = None
        self.beta = None
        
        super(InstantLayerNormalization, self).__init__()
        
        self.gamma = torch.ones(out_shape)
        self.gamma = nn.Parameter(self.gamma)
        
        self.beta = torch.zeros(out_shape)
        self.beta = nn.Parameter(self.beta)
        
    def forward(self, inps):
        mean = inps.mean(-1, keepdim=True)
        variance = torch.mean(torch.square(inps - mean), dim=-1, keepdim=True)
        std = torch.sqrt(variance + self.epsilon)
        
        outs = (inps - mean) / std
        print(outs.shape, self.gamma.shape)
        outs = outs * self.gamma
        outs = outs + self.beta
        
        return outs
    
class Multiply():
    def __init__(self):
        super(Multiply, self).__init__()
    
    def forward(self, ten1, ten2):
        mul_out = torch.mul(ten1, ten2)
        return mul_out

class NoiseReducer(nn.Module):
    def __init__(self, default_sr, n_fft, win_length, hop_len, dropout=0.5, batch_first=True, stride=2, normalized=False, bidir=False):
        
        self.default_sr = default_sr
        self.n_fft = n_fft
        self.win_length = win_length
        self.hop_len = hop_len
        self.normalized = normalized
        
        # LSTM UNITS
        self.rnn_dims = n_fft // 2 + 1
        self.num_layers = 2
        self.batch_first = True
        self.dropout = 0.2
        self.bidir = bidir
        
        # Dense Layer Units
        self.inp_dims = int(np.floor(((default_sr*sec) - (win_length - 1) - 1)/ hop_len + 5))#3))

        super(NoiseReducer, self).__init__()
        
        self.lstm1 = nn.LSTM(input_size=self.rnn_dims, hidden_size=self.rnn_dims, num_layers=self.num_layers, batch_first=self.batch_first, dropout=self.dropout, bidirectional=self.bidir)
        self.lstm2 = nn.LSTM(input_size=self.rnn_dims, hidden_size=self.rnn_dims, num_layers=self.num_layers, batch_first=self.batch_first, dropout=self.dropout, bidirectional=self.bidir)
        
        self.lstm3 = nn.LSTM(input_size=self.rnn_dims, hidden_size=self.rnn_dims, num_layers=self.num_layers, batch_first=self.batch_first, dropout=self.dropout, bidirectional=self.bidir)
        self.lstm4 = nn.LSTM(input_size=self.rnn_dims, hidden_size=self.rnn_dims, num_layers=self.num_layers, batch_first=self.batch_first, dropout=self.dropout, bidirectional=self.bidir)
        
        self.instantnorm1 = InstantLayerNormalization(self.rnn_dims, self.rnn_dims)
        self.instantnorm1 = InstantLayerNormalization(self.rnn_dims, self.rnn_dims)
        
        self.instancenorm1 = nn.InstanceNorm1d(self.rnn_dims)
        self.instancenorm2 = nn.InstanceNorm1d(self.rnn_dims)
        
        self.dense1 = nn.Linear(self.rnn_dims, self.rnn_dims)
        self.dense2 = nn.Linear(self.rnn_dims, self.rnn_dims)
        
        self.conv1 = nn.Conv1d(self.inp_dims, self.inp_dims, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(self.inp_dims, self.inp_dims, kernel_size=3, stride=1, padding=1)
                
    def stft_layer(self, sig):
        
        sig_stft = torch.stft(sig, n_fft=self.n_fft, hop_length=self.hop_len, win_length=self.win_length)
        
        sig_cplx = torch.view_as_complex(sig_stft)
        mag = sig_cplx.abs().permute(0, 2, 1)
        angle = sig_cplx.angle().permute(0, 2, 1)
        
        return [mag, angle]
    
    def istft_layer(self, mag, angle):
        mag = mag.permute(0, 2, 1)
        angle = angle.permute(0, 2, 1)
        mag = torch.unsqueeze(mag, dim=-1)
        angle = torch.unsqueeze(angle, dim=-1)
        pre_stft = torch.cat((mag, angle), dim=-1)
        stft_sig = torch.istft(pre_stft, n_fft=self.n_fft, win_length=self.win_length, hop_length=self.hop_len)
        
        return stft_sig
    

        
    def forward(self, inp_tensor):
        
        mag, angle = self.stft_layer(inp_tensor)
#         inp_tensor = inp_tensor.permute(0, 2, 1)
        mag_norm = self.instancenorm1(mag)
        x, hidden_states = self.lstm1(mag_norm)
        x, hidden_states = self.lstm2(x, hidden_states)
        mask = self.dense1(x)
        mask = F.sigmoid(mask)
        estimated_mag = torch.mul(mag, mask)
        
        feature_rep = self.conv1(estimated_mag)
        
        feature_norm = self.instancenorm2(feature_rep)
        x, hidden_states = self.lstm3(feature_norm)
        x, hidden_states = self.lstm4(x, hidden_states)
        mask = self.dense2(x)
        feature_mask = F.sigmoid(x)
        estimate_feat = torch.mul(feature_rep, feature_mask)
        
        estimate_frames = self.conv2(estimate_feat)
        estimate_sig = self.istft_layer(estimate_frames, angle)
        
        return estimate_sig
 
        
        

In [8]:
class Negative_SNR_Loss(nn.Module):
    def __init__(self):
        super(Negative_SNR_Loss, self).__init__()
    
    def forward(self, sig_pred, sig_true):
        
        sig_true_sq = torch.square(sig_true)
        sig_pred_sq = torch.square(sig_true - sig_pred)

        sig_true_mean = torch.mean(sig_true_sq)
        sig_pred_mean = torch.mean(sig_pred_sq)

        snr = sig_true_mean / sig_pred_mean + 1e-7
        loss = -1*torch.log10(snr)

        return loss

In [9]:
%%time
model = NoiseReducer(default_sr=default_sr, n_fft=n_fft, win_length=win_length, hop_len=hop_len).to(device)
model.to(device)

CPU times: user 1.18 s, sys: 639 ms, total: 1.82 s
Wall time: 2.53 s


NoiseReducer(
  (lstm1): LSTM(201, 201, num_layers=2, batch_first=True, dropout=0.2)
  (lstm2): LSTM(201, 201, num_layers=2, batch_first=True, dropout=0.2)
  (lstm3): LSTM(201, 201, num_layers=2, batch_first=True, dropout=0.2)
  (lstm4): LSTM(201, 201, num_layers=2, batch_first=True, dropout=0.2)
  (instantnorm1): InstantLayerNormalization()
  (instancenorm1): InstanceNorm1d(201, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (instancenorm2): InstanceNorm1d(201, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (dense1): Linear(in_features=201, out_features=201, bias=True)
  (dense2): Linear(in_features=201, out_features=201, bias=True)
  (conv1): Conv1d(961, 961, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(961, 961, kernel_size=(3,), stride=(1,), padding=(1,))
)

In [10]:
optim = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = Negative_SNR_Loss()

n_epochs=100

In [11]:
model.train()
train_loss = 0

for epoch in range(1, n_epochs+1):
    loop = tqdm(data_loader, leave=True, total=len(data_loader))
    for data, target in loop:
        
        data = data.to(device)
        target = target.to(device)
        
        optim.zero_grad()
#         with torch.cuda.amp.autocast():
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optim.step()
        
        loop.set_description(f"Epoch: [{epoch}/{n_epochs}]\t")
        loop.set_postfix(loss = loss.item())

KeyboardInterrupt: 

In [18]:
noise_add_sig, main_sig = signal_synthesis_dataset.__getitem__(10000)
noise_add_sig = torch.unsqueeze(noise_add_sig, dim=0).to(device)
main_sig = torch.unsqueeze(main_sig, dim=0).to(device)

In [19]:
with torch.no_grad():
    outs = model(noise_add_sig)

In [20]:
outs.shape

torch.Size([1, 96000])

In [22]:
x = outs[0].t().to("cpu").numpy()
ipd.Audio(x, rate=default_sr)

In [23]:
x = noise_add_sig[0].t().to("cpu").numpy()
ipd.Audio(x, rate=default_sr)

In [24]:
x = main_sig[0].t().to("cpu").numpy()
ipd.Audio(x, rate=default_sr)